In [24]:
# ========================================
# Step 1: Import libraries
# ========================================
import os
import json
import pandas as pd
from tqdm import tqdm   # progress bar

# ========================================
# Step 2: Load CE & BE data (original labels)
# ========================================
ce = pd.read_csv("CE.tsv", sep="\t")
be = pd.read_csv("BE.tsv", sep="\t")

# Add labels (1 = criminal, 0 = benign)
ce["label"] = 1
be["label"] = 0

# Create combined address spine
addr_spine = pd.concat([
    ce[["address", "label", "first_time", "last_time"]],
    be[["address", "label", "first_time", "last_time"]]
])

# Take a small sample of 250 addresses
spine_100 = pd.concat([
    addr_spine[addr_spine["label"] == 0].sample(n=250, random_state=42),
    addr_spine[addr_spine["label"] == 1].sample(n=250, random_state=42)
])

print(spine_100["label"].value_counts())
print("Sample spine created with 100 addresses")
print(spine_100.head())


label
0    250
1    250
Name: count, dtype: int64
Sample spine created with 100 addresses
                                         address  label        first_time  \
5438  0x569ec61f15aa4383aeaa8a001fea3b818b1e8d23      0   2022/11/14 2:08   
5405  0x561346b09b4b1c80b1404a884f9e58178a28ca84      0     2024/1/4 5:47   
88    0x0174a69fdecc5496224908ed3df2ff98df2f5056      0    2022/11/8 4:49   
5397  0x55fa2a35404f597b211119c83c56f2ce701e2598      0   2021/4/22 18:56   
1006  0x10940faad9e9fe68093b7f446d8fa446bb91a9c6      0  2022/10/10 16:20   

            last_time  
5438  2022/11/14 2:22  
5405   2024/3/12 5:47  
88    2024/3/26 14:52  
5397  2024/7/31 12:13  
1006   2023/3/26 7:13  


In [25]:
# ========================================
# Step 3: Define helper function to load transactions
# ========================================
def load_transactions_for_address(address, base_path="ETHData"):
    """
    Reads normalTransaction.json and ERC_20_transactions.json
    for a given wallet address.
    Returns a DataFrame of transactions.
    """
    addr_path = os.path.join(base_path, address)
    tx_list = []

    if not os.path.exists(addr_path):
        return pd.DataFrame()

    # Files we want to check
    files = ["normal_transactions.json", "ERC_20_transactions.json"]

    for file in files:
        fpath = os.path.join(addr_path, file)
        if os.path.exists(fpath):
            try:
                with open(fpath, "r") as f:
                    data = json.load(f)

                # Ensure list
                if isinstance(data, dict) and "result" in data:
                    data = data["result"]

                if isinstance(data, list):
                    for tx in data:
                        tx["address"] = address
                        tx["source_file"] = file
                        tx_list.append(tx)

            except Exception as e:
                print(f"Error reading {fpath}: {e}")

    return pd.DataFrame(tx_list)


In [26]:
# ========================================
# Step 4: Iterate through the 100 addresses and collect transactions
# ========================================
all_txs = []

for _, row in tqdm(spine_100.iterrows(), total=len(spine_100)):
    addr = row["address"]
    label = row["label"]

    df_addr = load_transactions_for_address(addr, base_path="ETHData")
    if not df_addr.empty:
        # Add labels and timestamps
        df_addr["label"] = label
        df_addr["first_time"] = row["first_time"]
        df_addr["last_time"] = row["last_time"]

        # --- Clean unwanted columns ---
        cols_to_drop = [
            "tokenName", "tokenSymbol", "tokenDecimal", 
            "input", "methodId", "contractAddress"
        ]
        df_addr = df_addr.drop(columns=[c for c in cols_to_drop if c in df_addr.columns], errors="ignore")

        # --- Remove transactions where functionName is missing ---
        # if "functionName" in df_addr.columns:
        #     df_addr = df_addr.dropna(subset=["functionName"])
        #     df_addr = df_addr[df_addr["functionName"].astype(str).str.strip() != ""]

        all_txs.append(df_addr)

# Combine all into one DataFrame
if all_txs:
    tx_df = pd.concat(all_txs, ignore_index=True)
else:
    tx_df = pd.DataFrame()

print("Final transaction DataFrame shape:", tx_df.shape)


100%|█████████████████████████████████████████| 500/500 [00:14<00:00, 34.44it/s]


Final transaction DataFrame shape: (285228, 22)


In [28]:
# ========================================
# Step 5: Save processed smaller dataset
# ========================================
if not tx_df.empty:
    tx_df.to_parquet("mini_transactions.parquet", index=False)
    tx_df.to_csv("mini_transactions.csv", index=False)
    print("Saved mini dataset (Parquet + CSV)")


Saved mini dataset (Parquet + CSV)


In [29]:
# ========================================
# Step 6: Inspect sample data
# ========================================
if not tx_df.empty:
    print(tx_df.head(5))
    print(tx_df.columns)
    print(tx_df["source_file"].value_counts())


  blockNumber                                          blockHash   timeStamp  \
0    15962797  0x99bba26b4a9e2ac8da2c0f6d011e695d0bf34488de5a...  1668362915   
1    15962869  0xc68fac12973278381f1440248f47460cf03ee61263ec...  1668363779   
2    18929595  0xd1b51e536ac50ce6cb08a74a032535e3af1e51a576c7...  1704318455   
3    18929610  0xe107cdfda0bf1992040a724d221c1bd6b2b35478deba...  1704318635   
4    18929625  0xc3710a3bd0cb6a8a735463cb84a9b618befa466fea59...  1704318815   

                                                hash    nonce  \
0  0xc1cc4b6e1d91420ba94689f42864e4c2b1dff7ed9e58...  3444505   
1  0xacfb9feb827c68c6fc3936f8a70970b473bc78b0c7b2...        0   
2  0xbda76e41324650172b19ffa3ab423254b26bef11cc70...       76   
3  0x5032f30c7bc2a99ea0ee23ed557193206032095c399d...        0   
4  0x3b75f0b627943b0472c7d982c3f6b44109d5d52b509e...       83   

  transactionIndex                                        from  \
0               69  0x9696f59e4d72e237be84ffd425dcad154bf96976

In [32]:
print(tx_df.columns.tolist())

['blockNumber', 'blockHash', 'timeStamp', 'hash', 'nonce', 'transactionIndex', 'from', 'to', 'value', 'gas', 'gasPrice', 'functionName', 'cumulativeGasUsed', 'txreceipt_status', 'gasUsed', 'confirmations', 'isError', 'address', 'source_file', 'label', 'first_time', 'last_time']


In [33]:
# Count null values per column
null_counts = tx_df.isnull().sum()

print("Null counts per column:")
print(null_counts)

Null counts per column:
blockNumber              0
blockHash                0
timeStamp                0
hash                     0
nonce                    0
transactionIndex         0
from                     0
to                       0
value                    0
gas                      0
gasPrice                 0
functionName         50315
cumulativeGasUsed        0
txreceipt_status     50315
gasUsed                  0
confirmations            0
isError              50315
address                  0
source_file              0
label                    0
first_time              32
last_time               32
dtype: int64


In [34]:
null_percentage = (tx_df.isnull().mean() * 100).round(2)

print("Null percentage per column:")
print(null_percentage)

Null percentage per column:
blockNumber           0.00
blockHash             0.00
timeStamp             0.00
hash                  0.00
nonce                 0.00
transactionIndex      0.00
from                  0.00
to                    0.00
value                 0.00
gas                   0.00
gasPrice              0.00
functionName         17.64
cumulativeGasUsed     0.00
txreceipt_status     17.64
gasUsed               0.00
confirmations         0.00
isError              17.64
address               0.00
source_file           0.00
label                 0.00
first_time            0.01
last_time             0.01
dtype: float64
